In [5]:
from supabase import Client, create_client
import os

def connect_to_supabase() -> Client:
    """Connect to Supabase database"""
    return create_client(
        os.environ.get("SUPABASE_URL"),
        os.environ.get("SUPABASE_KEY"),
    )


def connect_to_supabase_table(table_name: str) -> Client:
    return create_client(
        os.environ.get("SUPABASE_URL"),
        os.environ.get("SUPABASE_KEY"),
    ).table(table_name)


def get_metrics(user_email: str, project_name: str):
    table = connect_to_supabase_table("maintainability")
    return (
        table.select("*")
        .eq("user_email", user_email)
        .eq("project_name", project_name)
        .execute()
    )

In [6]:
response = get_metrics("test", "maintainability")

2023-10-17 16:50:09,019:INFO - HTTP Request: GET https://mgydwipshkixnewazzqb.supabase.co/rest/v1/maintainability?select=%2A&user_email=eq.test&project_name=eq.maintainability "HTTP/1.1 200 OK"


In [15]:
sessions = {}
for obj in response.data:
    sid = obj['session_id']
    if sid not in sessions:
        sessions[sid] = []
    sessions[sid].append(obj)

In [18]:
for session, objs in sessions.items():

    total_loc = 0
    for obj in objs:
        total_loc += obj['loc']

    scores = {
        'readability': 0,
        'design_quality': 0,
        'testability': 0,
        'consistency': 0,
        'debug_error_handling': 0,
    }
    for obj in objs:
        for key in scores:
            scores[key] += obj[key] * (obj['loc'] / total_loc)

    print(session, scores)

5c271ee8-da99-4618-9601-8800283e98d1 {'readability': 7.7227906976744185, 'design_quality': 6.826976744186046, 'testability': 7.62046511627907, 'consistency': 8.20093023255814, 'debug_error_handling': 7.606511627906975}
27f19dfa-b1ef-4101-8830-16400212ff2d {'readability': 7.710887465690758, 'design_quality': 6.813357731015555, 'testability': 7.619396157365052, 'consistency': 8.188472095150962, 'debug_error_handling': 7.634034766697164}
4d8ec281-f242-4224-9c5c-10435e32c59f {'readability': 7.734676742233418, 'design_quality': 6.828715365239296, 'testability': 7.675062972292192, 'consistency': 8.192275398824519, 'debug_error_handling': 7.588581024349287}
42ece357-4531-4922-8d49-10963a674a23 {'readability': 7.727272727272728, 'design_quality': 6.819000819000819, 'testability': 7.5782145782145784, 'consistency': 8.25962325962326, 'debug_error_handling': 7.645372645372647}
